In [29]:
import os
import random
from itertools import combinations
import numpy as np
import pandas as pd
from scipy.stats import pearsonr, spearmanr

pd.set_option('max_colwidth', None) # Setting to be able to view the entire comment

In [30]:
repo_dir = "/Users/ameliachu/repos/nlu-reddit-toxicity-dataset"
labelled_data_dir = f"{repo_dir}/data/labelled/"

In [31]:
labels = ['toxicity', 'severe_toxicity', 'identity_attack', 'insult', 'profanity', 'threat']
selected_columns = ['example_id', 'comment_for_evaluation'] + labels

all_labelled_df = []
for file_name in os.listdir(labelled_data_dir):
    labelled_df = pd.read_csv(f"{labelled_data_dir}{file_name}")[selected_columns]
    labelled_df['rater_id'] = file_name.split("_")[0]
    all_labelled_df.append(labelled_df)

In [32]:
labelled_df = pd.concat(all_labelled_df)

In [33]:
n_raters_lookup = labelled_df.groupby("example_id").rater_id.nunique().reset_index()

rater_ids = list(labelled_df.rater_id.unique())
num_raters = len(rater_ids)
random.shuffle(rater_ids)

all_rater_combinations = list(combinations(range(num_raters), 2))
print(rater_ids)
print(all_rater_combinations)

['yp2201', 'gm2858', 'ac4119', 'yj2369']
[(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]


In [34]:
labelled_df['rater_id'] = labelled_df['rater_id'].apply(lambda x: rater_ids.index(x))

In [35]:
example_ids_for_interrater = n_raters_lookup[n_raters_lookup['rater_id'] == 2]['example_id'].values

In [36]:
labelled_df_1 = labelled_df[labelled_df['example_id'].isin(example_ids_for_interrater)]

In [37]:
labelled_df_1[labelled_df_1['example_id']==587369].sort_values(by='example_id')

,example_id,comment_for_evaluation,toxicity,severe_toxicity,identity_attack,insult,profanity,threat,rater_id
126,587369,$BABA vs 🌈🐻 8-0,1.0,0.0,1.0,1.0,0.0,0.0,1
219,587369,$BABA vs 🌈🐻 8-0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [46]:
interrater_assessments_df = pd.melt(labelled_df_1,
                                    id_vars=['example_id','rater_id'], 
                                    value_vars=labels)\
                              .rename(columns={'value':'label'})

In [47]:
interrater_assessments_df = pd.pivot(interrater_assessments_df , 
                                     index=['variable', 'example_id'], 
                                     columns='rater_id', values='label').reset_index()

In [51]:
interrater_assessments_df['variable'].unique()

array(['identity_attack', 'insult', 'profanity', 'severe_toxicity',
       'threat', 'toxicity'], dtype=object)

In [ ]:
spearmanr_reliability = []

In [58]:
for rater_x, rater_y in all_rater_combinations:
    paired_assessment = interrater_assessments_df[(~interrater_assessments_df[rater_y].isnull()) & (~interrater_assessments_df[rater_x].isnull())]
    r_coef, p_value = spearmanr( paired_assessment[rater_x].values, paired_assessment[rater_y].values)
    data = {"rater_pair": (rater_x, rater_y), "r_coef": r_coef, "p_value": p_value, "spearmanr_reliability_type":"overall"}
    spearmanr_reliability.append(data)

    for variable in interrater_assessments_df['variable'].unique():
        print(variable)
        rater_x_values = paired_assessment[paired_assessment['variable']==variable][rater_x].values
        rater_y_values = paired_assessment[paired_assessment['variable']==variable][rater_y].values
        var_r_coef, var_p_value = spearmanr(rater_x_values, rater_y_values)
        data = {"rater_pair": (rater_x, rater_y), "r_coef": var_r_coef, "p_value": var_p_value, 
                "spearmanr_reliability_type":f"by_var_{variable}"}
        spearmanr_reliability.append(data)

identity_attack
insult
profanity
severe_toxicity
threat
toxicity
identity_attack
insult
profanity
severe_toxicity
threat
toxicity
identity_attack
insult
profanity
severe_toxicity
threat
toxicity
identity_attack
insult
profanity
severe_toxicity
threat
toxicity
identity_attack
insult
profanity
severe_toxicity
threat
toxicity
identity_attack
insult
profanity
severe_toxicity
threat
toxicity


/Users/ameliachu/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:4484: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


In [62]:
spearmanr_reliability_df = pd.DataFrame(spearmanr_reliability)[['spearmanr_reliability_type', 'rater_pair', 'r_coef','p_value']]

In [69]:
interrater_summary = spearmanr_reliability_df.groupby('spearmanr_reliability_type').agg({'r_coef':np.mean}).reset_index()
interrater_summary

,spearmanr_reliability_type,r_coef
0,by_var_identity_attack,0.812644
1,by_var_insult,0.795526
2,by_var_profanity,0.923831
3,by_var_severe_toxicity,0.780221
4,by_var_threat,1.000000
5,by_var_toxicity,0.781593
6,overall,0.881912


In [70]:
interrater_summary.columns = ["Attribute", "Reliability (Spearman R)"]

In [74]:
list(interrater_summary["Attribute"].values)

['by_var_identity_attack',
 'by_var_insult',
 'by_var_profanity',
 'by_var_severe_toxicity',
 'by_var_threat',
 'by_var_toxicity',
 'overall']

In [76]:
var_dict = {
    'by_var_identity_attack': 'Identity Attack',
    'by_var_insult': 'Insult',
    'by_var_profanity': 'Profanity',
    'by_var_severe_toxicity': 'Severe Toxicity',
    'by_var_threat': 'Threat',
    'by_var_toxicity': 'Toxicity',
    'overall': 'Overall'}

In [78]:
interrater_summary["Attribute"] = interrater_summary["Attribute"].apply(lambda x: var_dict[x])

In [82]:
interrater_summary.to_csv(f'{repo_dir}/reporting/assets/interrater-summary.csv', index=False)

In [81]:
interrater_summary

,Attribute,Reliability (Spearman R)
0,Identity Attack,0.812644
1,Insult,0.795526
2,Profanity,0.923831
3,Severe Toxicity,0.780221
4,Threat,1.000000
5,Toxicity,0.781593
6,Overall,0.881912
